# Query PDF Tutorial

In this tutorial, we demonstrate how to load a PDF and query it.

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/Yongqi099/evadb/blob/staging/apps/youtube_summary/summary.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" /> Run on Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/blob/master/tutorials/12-query-pdf.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" /> View source on GitHub</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/georgia-tech-db/eva/raw/master/tutorials/12-query-pdf.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" /> Download notebook</a>
  </td>
</table><br><br>

### Connect to EvaDB


In [2]:
%pip install --quiet "evadb[document,notebook]"
%pip install youtube_transcript_api
%pip install reportlab
%pip install gpt4all
%pip install pytube>=15.0.0

In [3]:
import evadb
cursor = evadb.connect().cursor()
import warnings
warnings.filterwarnings("ignore")

from google.colab import output
output.enable_custom_widget_manager()

from reportlab.platypus import SimpleDocTemplate, Paragraph
from reportlab.lib.styles import getSampleStyleSheet

from gpt4all import GPT4All

import pytube
from pytube import YouTube, extract  # noqa: E402
from youtube_transcript_api import YouTubeTranscriptApi

import os
import shutil
from typing import Dict, List

import pandas as pd

In [4]:
import logging

# Create a logger
logger = logging.getLogger(__name__)

# Set the level of this logger. This means that only logs of this level or higher will be logged.
logger.setLevel(logging.INFO)

# Create a file handler for outputting log messages to a file
handler = logging.FileHandler('logfile.log', mode='w')

# Create a formatter and add it to the handler
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
handler.setFormatter(formatter)

# Add the handler to the logger
logger.addHandler(handler)

In [5]:
gpt = GPT4All("ggml-model-gpt4all-falcon-q4_0.bin")

Found model file at  /root/.cache/gpt4all/ggml-model-gpt4all-falcon-q4_0.bin


In [20]:
MAX_CHUNK_SIZE = 200
DEFAULT_VIDEO_LINK = "https://www.youtube.com/watch?v=0E_wXecn4DU&pp=ygUKZGFpbHkgZG9zZQ%3D%3D"

# temporary file paths
# TRANSCRIPT_PATH = os.path.join("evadb_data", "tmp", "transcript.csv")
# SUMMARY_PATH = os.path.join("evadb_data", "tmp", "summary.csv")

TRANSCRIPT_PATH = "/content/transcript.csv"
SUMMARY_PATH = "summary.csv"


In [21]:
def receive_user_input() -> Dict:
    """Receives user input.

    Returns:
        user_input (dict): global configurations
    """
    print(
        "🔮 Welcome to EvaDB! This app lets you ask questions on any YouTube video.\nYou will only need to supply a Youtube URL.\n"
    )

    user_input = {"from_youtube": "y"}

    while True:
        # get Youtube video url
        video_link = str(input(
                "🌐 Enter the URL of the YouTube video (press Enter to use our default Youtube video URL): "
            ))

        if video_link == "":
            video_link = DEFAULT_VIDEO_LINK


        # Check if the URL is a valid YouTube URL
        if video_link.startswith("https://www.youtube.com/watch"):
            user_input["video_link"] = video_link
            break
        else:
            print("⚠️ Please enter a valid YouTube URL.")


    return user_input


def partition_transcript(raw_transcript: str):
    """Group video transcript elements when they are too large.

    Args:
        transcript (str): downloaded video transcript as a raw string.

    Returns:
        List: a list of partitioned transcript
    """
    if len(raw_transcript) <= MAX_CHUNK_SIZE:
        return [{"text": raw_transcript}]

    k = 2
    while True:
        if (len(raw_transcript) / k) <= MAX_CHUNK_SIZE:
            break
        else:
            k += 1
    chunk_size = int(len(raw_transcript) / k)

    partitioned_transcript = [
        {"text": raw_transcript[i : i + chunk_size]}
        for i in range(0, len(raw_transcript), chunk_size)
    ]
    if len(partitioned_transcript[-1]["text"]) < 30:
        partitioned_transcript.pop()
    return partitioned_transcript


def partition_summary(prev_summary: str):
    """Summarize a summary if a summary is too large.

    Args:
        prev_summary (str): previous summary that is too large.

    Returns:
        List: a list of partitioned summary
    """
    k = 2
    while True:
        if (len(prev_summary) / k) <= MAX_CHUNK_SIZE:
            break
        else:
            k += 1
    chunk_size = int(len(prev_summary) / k)

    new_summary = [
        {"summary": prev_summary[i : i + chunk_size]}
        for i in range(0, len(prev_summary), chunk_size)
    ]
    if len(new_summary[-1]["summary"]) < 30:
        new_summary.pop()
    return new_summary


def group_transcript(transcript: dict):
    """Group video transcript elements when they are too short.

    Args:
        transcript (dict): downloaded video transcript as a dictionary.

    Returns:
        str: full transcript as a single string.
    """
    new_line = ""
    for line in transcript:
        new_line += " " + line["text"]

    return new_line


def download_youtube_video_transcript(video_link: str):
    """Downloads a YouTube video's transcript.

    Args:
        video_link (str): url of the target YouTube video.
    """
    video_id = extract.video_id(video_link)
    print("⏳ Transcript download in progress...")
    transcript = YouTubeTranscriptApi.get_transcript(video_id)
    print("✅ Video transcript downloaded successfully.")
    return transcript

In [22]:
def generate_summary(cursor: evadb.EvaDBCursor):
    """Generate summary of a video transcript if it is too long (exceeds llm token limits)

    Args:
        cursor (EVADBCursor): evadb api cursor.
    """
    transcript_list = cursor.table("Transcript").select("text").df()["transcript.text"]
    if len(transcript_list) == 1:
        summary = transcript_list[0]
        df = pd.DataFrame([{"summary": summary}])
        df.to_csv(SUMMARY_PATH)

        cursor.drop_table("Summary", if_exists=True).execute()
        cursor.query(
            """CREATE TABLE IF NOT EXISTS Summary (summary TEXT(100));"""
        ).execute()
        cursor.load(SUMMARY_PATH, "Summary", "csv").execute()
        return

    generate_summary_rel = cursor.table("Transcript").select(
        "ChatGPT('summarize the video in detail', text)"
    )
    responses = generate_summary_rel.df()["chatgpt.response"]

    summary = ""
    for r in responses:
        summary += f"{r} \n"
    df = pd.DataFrame([{"summary": summary}])
    df.to_csv(SUMMARY_PATH)

    need_to_summarize = len(summary) > MAX_CHUNK_SIZE
    while need_to_summarize:
        partitioned_summary = partition_summary(summary)

        df = pd.DataFrame([{"summary": partitioned_summary}])
        df.to_csv(SUMMARY_PATH)

        cursor.drop_table("Summary", if_exists=True).execute()
        cursor.query(
            """CREATE TABLE IF NOT EXISTS Summary (summary TEXT(100));"""
        ).execute()
        cursor.load(SUMMARY_PATH, "Summary", "csv").execute()

        generate_summary_rel = cursor.table("Summary").select(
            "ChatGPT('summarize in detail', summary)"
        )
        responses = generate_summary_rel.df()["chatgpt.response"]
        summary = " ".join(responses)

        # no further summarization is needed if the summary is short enough
        if len(summary) <= MAX_CHUNK_SIZE:
            need_to_summarize = False

    # load final summary to table
    cursor.drop_table("Summary", if_exists=True).execute()
    cursor.query(
        """CREATE TABLE IF NOT EXISTS Summary (summary TEXT(100));"""
    ).execute()
    cursor.load(SUMMARY_PATH, "Summary", "csv").execute()


def generate_response(cursor: evadb.EvaDBCursor, question: str) -> str:
    """Generates question response with llm.

    Args:
        cursor (EVADBCursor): evadb api cursor.
        question (str): question to ask to llm.

    Returns
        str: response from llm.
    """

    if len(cursor.table("Transcript").select("text").df()["transcript.text"]) == 1:
        # Generate a response using the GPT4All model
        response = gpt.generate(question)
    else:
        # generate summary of the video if its too long
        if not os.path.exists(SUMMARY_PATH):
            generate_summary(cursor)

        # Generate a response using the GPT4All model and the summary
        summary = cursor.table("Summary").select("summary").df()["summary.summary"][0]
        prompt = summary + " " + question
        response = gpt.generate(prompt)

    return response

In [35]:
def cleanup():
    """Removes any temporary file / directory created by EvaDB."""
    if os.path.exists("evadb_data"):
        shutil.rmtree("evadb_data")


if __name__ == "__main__":
    # receive input from user
    user_input = receive_user_input()

    # load YouTube video transcript if it is available online
    transcript = None

    try:
        # establish evadb api cursor
        cursor = evadb.connect().cursor()

        if user_input["from_youtube"]:
            transcript = download_youtube_video_transcript(user_input["video_link"])

        raw_transcript_string = None



        logger.info('Info 1')

        # Group the list of transcripts into a single raw transcript.
        if transcript is not None:
            raw_transcript_string = group_transcript(transcript)

        logger.info('Info 2')

        # Partition the transcripts if they are too big to circumvent LLM token restrictions.
        if raw_transcript_string is not None:
            partitioned_transcript = partition_transcript(raw_transcript_string)
            df = pd.DataFrame(partitioned_transcript)
            df.to_csv(TRANSCRIPT_PATH)

        logger.info('Info 3')

        # load chunked transcript into table
        cursor.query("DROP TABLE IF EXISTS Transcript").df()

        logger.info('Info 4')

        # Load data into the 'Transcript' table from a CSV file
        # cursor.query("""CREATE TABLE IF NOT EXISTS Transcript (text TEXT(50));""").execute()
        cursor.query("LOAD CSV 'transcript.csv' INTO MyPDFs").df()

        logger.info('Info 5')
        cursor.load(TRANSCRIPT_PATH, "Transcript", "csv").execute()


        print("🪄 Ask anything about the video!")
        ready = True
        while ready:
            question = str(input("Question (enter 'exit' to exit): "))
            if question.lower() == "exit":
                ready = False
            else:
                # Generate response with chatgpt udf
                print("⏳ Generating response (may take a while)...")
                response = generate_response(cursor, question)
                print("✅ Answer:")
                print(response)

        cleanup()
        print("✅ Session ended.")
    except Exception as e:
        cleanup()
        print("❗️ Session ended with an error.")
        print(e)

🔮 Welcome to EvaDB! This app lets you ask questions on any YouTube video.
You will only need to supply a Youtube URL.

🌐 Enter the URL of the YouTube video (press Enter to use our default Youtube video URL): 
⏳ Transcript download in progress...


INFO:__main__:Info 1
INFO:__main__:Info 2
INFO:__main__:Info 3
INFO:__main__:Info 4
10-11-2023 18:53:21 ERROR [load_csv_executor:load_csv_executor.py:exec:0048] MyPDFs does not exist.
ERROR:evadb.utils.logging_manager:MyPDFs does not exist.
10-11-2023 18:53:21 ERROR [plan_executor:plan_executor.py:execute_plan:0186] MyPDFs does not exist.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/plan_executor.py", line 182, in execute_plan
    yield from output
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/load_executor.py", line 48, in exec
    for batch in executor.exec():
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/load_csv_executor.py", line 49, in exec
    raise ExecutorError(error)
evadb.executor.executor_utils.ExecutorError: MyPDFs does not exist.
ERROR:evadb.utils.logging_manager:MyPDFs does not exist.
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/evadb/executor/plan

✅ Video transcript downloaded successfully.
❗️ Session ended with an error.
MyPDFs does not exist.


### Video Link

In [24]:
# replace with your video URL
video_url = 'https://www.youtube.com/watch?v=0E_wXecn4DU&pp=ygUKZGFpbHkgZG9zZQ%3D%3D'

### Get Youtube Transcript

In [25]:
# Check if the video URL starts with 'https://www.youtube.com/watch'
if not video_url.startswith('https://www.youtube.com/watch'):
    raise ValueError("Invalid video URL. It should start with 'https://www.youtube.com/watch'")

# Extract the video ID from the URL
video_id = video_url.split('=')[1]

# Get the transcript of the video
transcript = YouTubeTranscriptApi.get_transcript(video_id)

In [26]:
# Combine all the parts into a single string
full_transcript = " ".join([part['text'] for part in transcript])
# Write the transcript to a text file
with open('transcript.txt', 'w') as f:
    f.write(full_transcript)

# Write the transcript to a PDF file
doc = SimpleDocTemplate("pdf_sample1.pdf")
styles = getSampleStyleSheet()
story = [Paragraph(full_transcript, styles["BodyText"])]
doc.build(story)

In [27]:
# Prompt the model to summarize the transcript
prompt = "Summarize the following text: " + full_transcript
summary = gpt.generate(prompt)


In [28]:
print(len(full_transcript))
print(len(summary))
print(summary)

995
0



### Hugging Face Summary

Load PDFs

In [36]:
!wget -nc "https://www.dropbox.com/s/fv6pqdneth3l6fz/pdf_sample1.pdf"

File ‘pdf_sample1.pdf’ already there; not retrieving.



In [37]:
cursor.query("DROP TABLE IF EXISTS MyPDFs").df()
cursor.query("LOAD PDF 'pdf_sample1.pdf' INTO MyPDFs").df()

10-11-2023 18:57:28 ERROR [plan_executor:plan_executor.py:execute_plan:0186] (sqlite3.OperationalError) unable to open database file
(Background on this error at: https://sqlalche.me/e/20/e3q8)
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 145, in __init__
    self._dbapi_connection = engine.raw_connection()
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/engine/base.py", line 3288, in raw_connection
    return self.pool.connect()
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 452, in connect
    return _ConnectionFairy._checkout(self)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 1267, in _checkout
    fairy = _ConnectionRecord.checkout(pool)
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/base.py", line 716, in checkout
    rec = pool._do_get()
  File "/usr/local/lib/python3.10/dist-packages/sqlalchemy/pool/impl.py", line 16

ExecutorError: ignored

Retrieve Text from Loaded PDFs

In [ ]:
cursor.query("SELECT * FROM MyPDFs").df()

In [ ]:
cursor.query("""
    SELECT *
    FROM MyPDFs
""").df()

Create UDFs for Text Classification and Text Summarization

In [ ]:
cursor.query("""
    CREATE FUNCTION IF NOT EXISTS TextClassifier
    TYPE HuggingFace
    TASK 'text-classification'
    MODEL 'distilbert-base-uncased-finetuned-sst-2-english'
""").df()

In [ ]:
cursor.query("""
    CREATE FUNCTION IF NOT EXISTS TextSummarizer
    TYPE HuggingFace
    TASK 'summarization'
    MODEL 'facebook/bart-large-cnn'
""").df()

Get Summaries of a Subset of Paragraphs with Negative Sentiment

In [ ]:
cursor.query("""
    SELECT data, TextSummarizer(data)
    FROM MyPDFs
""").df()
